In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [2]:
# Define the LEMCell
class LEMCell(nn.Module):
    def __init__(self, ninp, nhid, dt):
        super(LEMCell, self).__init__()
        self.ninp = ninp
        self.nhid = nhid
        self.dt = dt
        self.inp2hid = nn.Linear(ninp, 4 * nhid)
        self.hid2hid = nn.Linear(nhid, 3 * nhid)
        self.transform_z = nn.Linear(nhid, nhid)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.nhid)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, y, z):
        transformed_inp = self.inp2hid(x)
        transformed_hid = self.hid2hid(y)
        i_dt1, i_dt2, i_z, i_y = transformed_inp.chunk(4, 1)
        h_dt1, h_dt2, h_y = transformed_hid.chunk(3, 1)

        ms_dt_bar = self.dt * torch.sigmoid(i_dt1 + h_dt1)
        ms_dt = self.dt * torch.sigmoid(i_dt2 + h_dt2)

        z = (1. - ms_dt) * z + ms_dt * torch.tanh(i_y + h_y)
        y = (1. - ms_dt_bar) * y + ms_dt_bar * torch.tanh(self.transform_z(z) + i_z)

        return y, z

# Define the LEM model
class LEM(nn.Module):
    def __init__(self, ninp, nhid, nout, dt=1.):
        super(LEM, self).__init__()
        self.nhid = nhid
        self.cell = LEMCell(ninp, nhid, dt)
        self.classifier = nn.Linear(nhid, nout)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'classifier' in name and 'weight' in name:
                nn.init.kaiming_normal_(param.data)

    def forward(self, input):
        y = input.data.new(input.size(1), self.nhid).zero_()
        z = input.data.new(input.size(1), self.nhid).zero_()
        for x in input:
            y, z = self.cell(x, y, z)
        out = self.classifier(y)
        return out


### PINN data importing

In [3]:
# importing data

# Load the .mat file
mat_data = scipy.io.loadmat('burg.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u = mat_data['u1']



### Exact Solution data importing

In [4]:
# importing data

import torch
import torch.nn as nn
import numpy as np
import time
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('burgers_shock.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u_1 = mat_data['usol']


In [5]:
# Set random seed for reproducibility
#torch.manual_seed(42)

# Toy problem data
input_size = 256
hidden_size = 32
output_size = 256
sequence_length = 79
batch_size = 1
num_epochs = 20000

# Set random seed for reproducibility
#torch.manual_seed(42)
u[:, 0:100].shape

(256, 100)

In [6]:
input_data = u[:,0:79]
target_data = u[:,1:80]

test_data = u[:,79]
#test_target = u[:,80:100]

print("test data shape", test_data.shape)
#print("test target shape", test_target.shape)

print("input data shape",input_data.shape)
print("Target data shape",target_data.shape)

# Convert data to tensors
input_tensor = torch.tensor(input_data.T).view(batch_size, sequence_length, input_size).float()
target_tensor = torch.tensor(target_data.T).view(batch_size, sequence_length, output_size).float()

print("input tensor shape",input_tensor.shape)
print("Target tensor shape",target_tensor.shape)

test data shape (256,)
input data shape (256, 79)
Target data shape (256, 79)
input tensor shape torch.Size([1, 79, 256])
Target tensor shape torch.Size([1, 79, 256])


In [7]:
# Convert test data to tensors
test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()
#test_target_tensor = torch.tensor(test_target.T).view(batch_size, 20, output_size).float()
target_tensor = torch.squeeze(target_tensor)

In [8]:
# Create LEM instance
lem = LEM(input_size, hidden_size, output_size, dt=0.3)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lem.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    output = lem(input_tensor)
    loss = criterion(output, target_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.16f}')

Epoch: 10/20000, Loss: 0.3723836243152618
Epoch: 20/20000, Loss: 0.3186952471733093
Epoch: 30/20000, Loss: 0.2613539397716522
Epoch: 40/20000, Loss: 0.2024728804826736
Epoch: 50/20000, Loss: 0.1497684866189957
Epoch: 60/20000, Loss: 0.1076712161302567
Epoch: 70/20000, Loss: 0.0766476094722748
Epoch: 80/20000, Loss: 0.0554979890584946
Epoch: 90/20000, Loss: 0.0421221479773521
Epoch: 100/20000, Loss: 0.0340989194810390
Epoch: 110/20000, Loss: 0.0292407870292664
Epoch: 120/20000, Loss: 0.0261520277708769
Epoch: 130/20000, Loss: 0.0240727886557579
Epoch: 140/20000, Loss: 0.0225642379373312
Epoch: 150/20000, Loss: 0.0213899798691273
Epoch: 160/20000, Loss: 0.0204122308641672
Epoch: 170/20000, Loss: 0.0195362363010645
Epoch: 180/20000, Loss: 0.0187038127332926
Epoch: 190/20000, Loss: 0.0178971346467733
Epoch: 200/20000, Loss: 0.0170822050422430
Epoch: 210/20000, Loss: 0.0162416473031044
Epoch: 220/20000, Loss: 0.0154027044773102
Epoch: 230/20000, Loss: 0.0145623609423637
Epoch: 240/20000, Lo

Epoch: 2020/20000, Loss: 0.0001747199858073
Epoch: 2030/20000, Loss: 0.0001728378556436
Epoch: 2040/20000, Loss: 0.0001722495944705
Epoch: 2050/20000, Loss: 0.0001732510572765
Epoch: 2060/20000, Loss: 0.0001676844112808
Epoch: 2070/20000, Loss: 0.0001666480820859
Epoch: 2080/20000, Loss: 0.0001638687244849
Epoch: 2090/20000, Loss: 0.0001622217096156
Epoch: 2100/20000, Loss: 0.0001605589932296
Epoch: 2110/20000, Loss: 0.0001589059829712
Epoch: 2120/20000, Loss: 0.0001573022309458
Epoch: 2130/20000, Loss: 0.0001557369105285
Epoch: 2140/20000, Loss: 0.0001542125974083
Epoch: 2150/20000, Loss: 0.0001527155545773
Epoch: 2160/20000, Loss: 0.0001512546441518
Epoch: 2170/20000, Loss: 0.0001500795187894
Epoch: 2180/20000, Loss: 0.0001640897535253
Epoch: 2190/20000, Loss: 0.0001508787245257
Epoch: 2200/20000, Loss: 0.0001470756251365
Epoch: 2210/20000, Loss: 0.0001444886293029
Epoch: 2220/20000, Loss: 0.0001433460856788
Epoch: 2230/20000, Loss: 0.0001418323372491
Epoch: 2240/20000, Loss: 0.00014

Epoch: 3890/20000, Loss: 0.0000631739385426
Epoch: 3900/20000, Loss: 0.0000659555953462
Epoch: 3910/20000, Loss: 0.0000660851073917
Epoch: 3920/20000, Loss: 0.0000689640874043
Epoch: 3930/20000, Loss: 0.0000641512669972
Epoch: 3940/20000, Loss: 0.0000623614396318
Epoch: 3950/20000, Loss: 0.0000617215264356
Epoch: 3960/20000, Loss: 0.0000613492011325
Epoch: 3970/20000, Loss: 0.0000610276583757
Epoch: 3980/20000, Loss: 0.0000607124638918
Epoch: 3990/20000, Loss: 0.0000604095912422
Epoch: 4000/20000, Loss: 0.0000601257306698
Epoch: 4010/20000, Loss: 0.0000598456172156
Epoch: 4020/20000, Loss: 0.0000595631245233
Epoch: 4030/20000, Loss: 0.0000592818178120
Epoch: 4040/20000, Loss: 0.0000589998999203
Epoch: 4050/20000, Loss: 0.0000587178619753
Epoch: 4060/20000, Loss: 0.0000584355511819
Epoch: 4070/20000, Loss: 0.0000581529420742
Epoch: 4080/20000, Loss: 0.0000578700346523
Epoch: 4090/20000, Loss: 0.0000575868471060
Epoch: 4100/20000, Loss: 0.0000573034121771
Epoch: 4110/20000, Loss: 0.00005

Epoch: 5910/20000, Loss: 0.0000167793768924
Epoch: 5920/20000, Loss: 0.0000166593163158
Epoch: 5930/20000, Loss: 0.0000165408491739
Epoch: 5940/20000, Loss: 0.0000164841057995
Epoch: 5950/20000, Loss: 0.0000312253250740
Epoch: 5960/20000, Loss: 0.0000419243006036
Epoch: 5970/20000, Loss: 0.0000235428688029
Epoch: 5980/20000, Loss: 0.0000197194713110
Epoch: 5990/20000, Loss: 0.0000173694606929
Epoch: 6000/20000, Loss: 0.0000162872256624
Epoch: 6010/20000, Loss: 0.0000158624134201
Epoch: 6020/20000, Loss: 0.0000156349233293
Epoch: 6030/20000, Loss: 0.0000154803346959
Epoch: 6040/20000, Loss: 0.0000153567525558
Epoch: 6050/20000, Loss: 0.0000152482880367
Epoch: 6060/20000, Loss: 0.0000151449639816
Epoch: 6070/20000, Loss: 0.0000150412442963
Epoch: 6080/20000, Loss: 0.0000149391953528
Epoch: 6090/20000, Loss: 0.0000148383023770
Epoch: 6100/20000, Loss: 0.0000147384653246
Epoch: 6110/20000, Loss: 0.0000146396278069
Epoch: 6120/20000, Loss: 0.0000145418152897
Epoch: 6130/20000, Loss: 0.00001

Epoch: 7800/20000, Loss: 0.0000071770687100
Epoch: 7810/20000, Loss: 0.0000071561639743
Epoch: 7820/20000, Loss: 0.0000071352778832
Epoch: 7830/20000, Loss: 0.0000071144431786
Epoch: 7840/20000, Loss: 0.0000070936343946
Epoch: 7850/20000, Loss: 0.0000070728583523
Epoch: 7860/20000, Loss: 0.0000070520918598
Epoch: 7870/20000, Loss: 0.0000070313276410
Epoch: 7880/20000, Loss: 0.0000070105725172
Epoch: 7890/20000, Loss: 0.0000069898292168
Epoch: 7900/20000, Loss: 0.0000069690736382
Epoch: 7910/20000, Loss: 0.0000069483139669
Epoch: 7920/20000, Loss: 0.0000069276302384
Epoch: 7930/20000, Loss: 0.0000069123875619
Epoch: 7940/20000, Loss: 0.0000077108443293
Epoch: 7950/20000, Loss: 0.0000787494573160
Epoch: 7960/20000, Loss: 0.0000125194392240
Epoch: 7970/20000, Loss: 0.0000093887265393
Epoch: 7980/20000, Loss: 0.0000078912798926
Epoch: 7990/20000, Loss: 0.0000070950873123
Epoch: 8000/20000, Loss: 0.0000068504523369
Epoch: 8010/20000, Loss: 0.0000067835180744
Epoch: 8020/20000, Loss: 0.00000

...

Epoch: 11660/20000, Loss: 0.0000018964115043
Epoch: 11670/20000, Loss: 0.0000018899222596
Epoch: 11680/20000, Loss: 0.0000018839898530
Epoch: 11690/20000, Loss: 0.0000018782208144
Epoch: 11700/20000, Loss: 0.0000018725625068
Epoch: 11710/20000, Loss: 0.0000018669578594
Epoch: 11720/20000, Loss: 0.0000018614194914
Epoch: 11730/20000, Loss: 0.0000018559343289
Epoch: 11740/20000, Loss: 0.0000018504871377
Epoch: 11750/20000, Loss: 0.0000018450830339
Epoch: 11760/20000, Loss: 0.0000018397214490
Epoch: 11770/20000, Loss: 0.0000018344538830
Epoch: 11780/20000, Loss: 0.0000018343695274
Epoch: 11790/20000, Loss: 0.0000027176436106
Epoch: 11800/20000, Loss: 0.0000525335126440
Epoch: 11810/20000, Loss: 0.0000110976334327
Epoch: 11820/20000, Loss: 0.0000049909644986
Epoch: 11830/20000, Loss: 0.0000025536432986
Epoch: 11840/20000, Loss: 0.0000019900019197
Epoch: 11850/20000, Loss: 0.0000018286065142
Epoch: 11860/20000, Loss: 0.0000018241031512
Epoch: 11870/20000, Loss: 0.0000018155253656
Epoch: 118

Epoch: 13530/20000, Loss: 0.0000013715920204
Epoch: 13540/20000, Loss: 0.0000014250174445
Epoch: 13550/20000, Loss: 0.0000014060399280
Epoch: 13560/20000, Loss: 0.0000013597796169
Epoch: 13570/20000, Loss: 0.0000013425224097
Epoch: 13580/20000, Loss: 0.0000013503375840
Epoch: 13590/20000, Loss: 0.0000013394468397
Epoch: 13600/20000, Loss: 0.0000013410067368
Epoch: 13610/20000, Loss: 0.0000013494948234
Epoch: 13620/20000, Loss: 0.0000015542874507
Epoch: 13630/20000, Loss: 0.0000087476200861
Epoch: 13640/20000, Loss: 0.0000015296765241
Epoch: 13650/20000, Loss: 0.0000036592120978
Epoch: 13660/20000, Loss: 0.0000025105987334
Epoch: 13670/20000, Loss: 0.0000014015860188
Epoch: 13680/20000, Loss: 0.0000013526222347
Epoch: 13690/20000, Loss: 0.0000013481594578
Epoch: 13700/20000, Loss: 0.0000013328453861
Epoch: 13710/20000, Loss: 0.0000013204215747
Epoch: 13720/20000, Loss: 0.0000013171704722
Epoch: 13730/20000, Loss: 0.0000013161909465
Epoch: 13740/20000, Loss: 0.0000013120164795
Epoch: 137

Epoch: 15380/20000, Loss: 0.0000020334352939
Epoch: 15390/20000, Loss: 0.0000018962007289
Epoch: 15400/20000, Loss: 0.0000011578355270
Epoch: 15410/20000, Loss: 0.0000012131711173
Epoch: 15420/20000, Loss: 0.0000011069911352
Epoch: 15430/20000, Loss: 0.0000011126600157
Epoch: 15440/20000, Loss: 0.0000011029236475
Epoch: 15450/20000, Loss: 0.0000010974074485
Epoch: 15460/20000, Loss: 0.0000010947699138
Epoch: 15470/20000, Loss: 0.0000010929368273
Epoch: 15480/20000, Loss: 0.0000010912516473
Epoch: 15490/20000, Loss: 0.0000010896204685
Epoch: 15500/20000, Loss: 0.0000010884153880
Epoch: 15510/20000, Loss: 0.0000011017256156
Epoch: 15520/20000, Loss: 0.0000021665268832
Epoch: 15530/20000, Loss: 0.0000155404795805
Epoch: 15540/20000, Loss: 0.0000025133074359
Epoch: 15550/20000, Loss: 0.0000016408151851
Epoch: 15560/20000, Loss: 0.0000016333848407
Epoch: 15570/20000, Loss: 0.0000012095113107
Epoch: 15580/20000, Loss: 0.0000010922366300
Epoch: 15590/20000, Loss: 0.0000011163650697
Epoch: 156

Epoch: 17220/20000, Loss: 0.0000019622773380
Epoch: 17230/20000, Loss: 0.0000014661692376
Epoch: 17240/20000, Loss: 0.0000011922990097
Epoch: 17250/20000, Loss: 0.0000009471293652
Epoch: 17260/20000, Loss: 0.0000009351663834
Epoch: 17270/20000, Loss: 0.0000009162932315
Epoch: 17280/20000, Loss: 0.0000009107487244
Epoch: 17290/20000, Loss: 0.0000009043381510
Epoch: 17300/20000, Loss: 0.0000009024349197
Epoch: 17310/20000, Loss: 0.0000009083997270
Epoch: 17320/20000, Loss: 0.0000011597778666
Epoch: 17330/20000, Loss: 0.0000141397267726
Epoch: 17340/20000, Loss: 0.0000076029400589
Epoch: 17350/20000, Loss: 0.0000016660193296
Epoch: 17360/20000, Loss: 0.0000019132273792
Epoch: 17370/20000, Loss: 0.0000010038826304
Epoch: 17380/20000, Loss: 0.0000010065155038
Epoch: 17390/20000, Loss: 0.0000009313089322
Epoch: 17400/20000, Loss: 0.0000008982401027
Epoch: 17410/20000, Loss: 0.0000008953043675
Epoch: 17420/20000, Loss: 0.0000008924193935
Epoch: 17430/20000, Loss: 0.0000008897387147
Epoch: 174

Epoch: 19220/20000, Loss: 0.0000007245807865
Epoch: 19230/20000, Loss: 0.0000007232218309
Epoch: 19240/20000, Loss: 0.0000007216282256
Epoch: 19250/20000, Loss: 0.0000007201002745
Epoch: 19260/20000, Loss: 0.0000007186721405
Epoch: 19270/20000, Loss: 0.0000007173740642
Epoch: 19280/20000, Loss: 0.0000007161379472
Epoch: 19290/20000, Loss: 0.0000007158712947
Epoch: 19300/20000, Loss: 0.0000007667448472
Epoch: 19310/20000, Loss: 0.0000059262779359
Epoch: 19320/20000, Loss: 0.0000051040060498
Epoch: 19330/20000, Loss: 0.0000017696470422
Epoch: 19340/20000, Loss: 0.0000017464599296
Epoch: 19350/20000, Loss: 0.0000010580348544
Epoch: 19360/20000, Loss: 0.0000008131853519
Epoch: 19370/20000, Loss: 0.0000007264100077
Epoch: 19380/20000, Loss: 0.0000007332649830
Epoch: 19390/20000, Loss: 0.0000007105257964
Epoch: 19400/20000, Loss: 0.0000007104301858
Epoch: 19410/20000, Loss: 0.0000007066617513
Epoch: 19420/20000, Loss: 0.0000007064429610
Epoch: 19430/20000, Loss: 0.0000007564366342
Epoch: 194

In [9]:
print(test_tensor.shape)
prediction_tensor = torch.zeros(1, 20, 256).float()
print(prediction_tensor.shape)

torch.Size([1, 1, 256])
torch.Size([1, 20, 256])


In [10]:
with torch.no_grad():
    prediction = lem(test_tensor)
    prediction = prediction.view(1, 1, 256).float()
    prediction_tensor[:, 0, :] = prediction
    for i in range(19):
        prediction = lem(prediction)
        prediction = prediction.view(1, 1, 256).float()
        prediction_tensor[:, i+1, :] = prediction
        

### Four different types of error

In [11]:
# Exact Solution

u_test = u_1.T
u_test_full = u_test[80:100, :]

In [12]:
prediction_tensor = torch.squeeze(prediction_tensor)


In [13]:
# Extrapolation

k1 = ( prediction_tensor - u_test_full)**2
u_test_full_tensor = torch.tensor(u_test_full**2)
u_test_full_tensor.shape

torch.Size([20, 256])

### L^2 norm error

In [14]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(k1)/ torch.mean(u_test_full_tensor)

print("Relative Error Test: ", relative_error_test.item(), "%")

Relative Error Test:  0.061881364236231876 %


In [15]:
2+

SyntaxError: invalid syntax (4209523232.py, line 1)

### Max absolute norm error

In [ ]:
R_abs = torch.max(torch.abs(prediction_tensor - u_test_full))

In [ ]:
print(R_abs)

### Explained variance score

In [ ]:
import torch

a = prediction_tensor
b = u_test_full
# Assuming 'a' is your predicted values (model's predictions) and 'b' is the true values (ground truth)
# Make sure 'a' and 'b' are PyTorch tensors
# a = torch.tensor(a)
b = torch.tensor(b)
# Calculate the mean of 'b'
mean_b = torch.mean(b)

# Calculate the Explained Variance Score
numerator = torch.var(b - a)  # Variance of the differences between 'b' and 'a'
denominator = torch.var(b)    # Variance of 'b'
evs = 1 - numerator / denominator

print("Explained Variance Score:", evs.item())

### Mean absolute error

In [ ]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(torch.abs(prediction_tensor - u_test_full))

print("Relative Error Test: ", relative_error_test, "%")

### Contour plot for PINN (80 percent) and (20 percentage lem prediction)

In [ ]:
print(prediction_tensor.shape)
prediction_tensor = torch.squeeze(prediction_tensor)
input_tensor = torch.squeeze(input_tensor)

conc_u = torch.squeeze(input_tensor)
concatenated_tensor = torch.cat((conc_u, prediction_tensor), dim=0)

x1 = np.linspace(-1, 1, 256)
t1 = np.linspace(0, 1, 99)

### Snapshot time plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed




final_time_output = prediction_tensor[3, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, 83].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.83}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.83_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.83_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed



final_time_output = prediction_tensor[-2, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, -2].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.98}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.98_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.98_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


### Contour plot where 80 percent for PINN solution and 20 percent for lem solution

### Exact contour

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = u_1.T

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

#plt.savefig('Contour_Exact.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_exact.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = concatenated_tensor.numpy()

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t1)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

# Add a dotted line at t = 0.8
plt.axvline(x=0.8, color='black', linestyle='dotted', linewidth=5)

#plt.savefig('Contour_LEM_20.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_LEM_20.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()
